In [1]:
import numpy as np
import pandas as pd
import datetime
import pickle
import os

from sklearn.utils import class_weight
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam, SGD
from keras.regularizers import l2
from keras.utils import to_categorical

In [2]:
root_path = '.'

## Google Colab

In [3]:
try:
    from google.colab import drive
    drive.mount('/content/drive')

    root_path = '/content/drive/MyDrive/projects/facial_feeling_detection'
except:
    pass

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##### Check GPU

In [4]:
try:
    from tensorflow.python.client import device_lib

    device_lib.list_local_devices()
except:
    pass

## Constants

In [5]:
SEED = 42
LABELS = {0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'neutral', 5: 'sad', 6: 'surprise'}
TRAIN_PATH = root_path + '/data/train/'
TEST_PATH = root_path + '/data/test/'
MODELS_PATH = root_path + '/models/'
SUBMISSIONS_PATH = root_path + '/submissions/'

## Data Loading

In [6]:
data = np.load(root_path + '/data/unbalanced_data.npz')
X_train = data['X_train']
X_test = data['X_test']
y_train = data['y_train']

with open(root_path + '/data/test_ids.pkl', 'rb') as f:
    test_ids = pickle.load(f)

submission_ids = pd.read_csv(root_path + '/data/test_set.csv')

## Model

In [7]:
batch_size = 64
learning_rate = .0001 # .01
kernel_regularizer = .01 # .01
dropout_conv = .3
dropout_dense = .5

In [8]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1), kernel_regularizer=l2(kernel_regularizer)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(dropout_conv),

    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(kernel_regularizer)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(dropout_conv),

    Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(kernel_regularizer)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(dropout_conv),

    Conv2D(256, (3, 3), activation='relu', kernel_regularizer=l2(kernel_regularizer)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(dropout_conv),

    Flatten(),

    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(dropout_dense),

    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(dropout_dense),
    
    Dense(7, activation='softmax')
])

In [9]:
model.compile(
    optimizer=Adam(learning_rate=learning_rate),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# print(model.summary())

In [10]:
callback_path = MODELS_PATH + 'callback.h5'

earlystop = EarlyStopping(patience=7)
mcheckpoint = ModelCheckpoint(
    callback_path,
    monitor='val_accuracy',
    save_best_only=True
)

##### Weights

In [11]:
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = {label: weight for label, weight in zip(np.unique(y_train), class_weights)}

##### To Categorical

In [12]:
y_train = to_categorical(y_train, num_classes=7)

##### Train

In [20]:
history = model.fit(
    X_train, y_train,
    validation_split=.2,
    epochs=100,
    batch_size=batch_size,
    class_weight=class_weight_dict,
    # callbacks=[earlystop, mcheckpoint]
    callbacks=[mcheckpoint]
)

Epoch 1/100
361/361 [==============================] - 5s 15ms/step - loss: 1.2302 - accuracy: 0.6192 - val_loss: 1.4013 - val_accuracy: 0.5934
Epoch 2/100
361/361 [==============================] - 5s 13ms/step - loss: 1.2195 - accuracy: 0.6319 - val_loss: 1.3979 - val_accuracy: 0.6052
Epoch 3/100
361/361 [==============================] - 5s 13ms/step - loss: 1.2084 - accuracy: 0.6287 - val_loss: 1.4841 - val_accuracy: 0.5818
Epoch 4/100
361/361 [==============================] - 5s 14ms/step - loss: 1.2002 - accuracy: 0.6299 - val_loss: 1.4059 - val_accuracy: 0.5979
Epoch 5/100
361/361 [==============================] - 5s 13ms/step - loss: 1.2064 - accuracy: 0.6299 - val_loss: 1.4300 - val_accuracy: 0.5886
Epoch 6/100
361/361 [==============================] - 5s 14ms/step - loss: 1.2009 - accuracy: 0.6347 - val_loss: 1.3841 - val_accuracy: 0.6069
Epoch 7/100
361/361 [==============================] - 5s 14ms/step - loss: 1.2194 - accuracy: 0.6285 - val_loss: 1.4104 - val_accuracy:

In [14]:
max_val_accuracy = max(history.history['val_accuracy'])
date = datetime.datetime.now().strftime('%y%m%d%H%M%S')
model_path = f'{MODELS_PATH}model_val_acc_{max_val_accuracy:.4f}_{date}.h5'

os.rename(callback_path, model_path)

## Prediction/Submission

In [15]:
# model_path = MODELS_PATH + 'model_val_acc_0.4901_230224172910.h5'

model_last = load_model(model_path)

In [16]:
y_pred = model_last.predict(X_test)
predictions = [np.argmax(pred) for pred in y_pred]

221/221 [==============================] - 1s 3ms/step


In [17]:
submission_pred = pd.DataFrame(test_ids, columns=['id_img']).astype(int)
submission_pred['label'] = [LABELS[pred] for pred in predictions]
submission_pred = pd.merge(submission_ids, submission_pred)

In [18]:
submission_path = model_path.replace(MODELS_PATH, SUBMISSIONS_PATH).replace('model_', 'submission_').replace('.h5', '.csv')
submission_pred.to_csv(submission_path, index=False)